In [1]:
library(PQLseq)

In [2]:
fam = read.table('../gemma/plink_files/16p12_RNA_by_rep.fam', sep=' ')

In [3]:
pheno = read.table('../16p12.2_rnaseq_analysis/data/pheno_final.tsv', 
                   sep='\t', header=TRUE, stringsAsFactors = F)
rownames(pheno) = pheno$samp

In [4]:
counts = read.table('../16p12.2_rnaseq_analysis/data/16p12_lcl_gene_reads_underscores.gct',
                   sep='\t', header=T)
rownames(counts) = counts$Name
counts$Name = NULL

gencode2ensembl = function(s) {
    return(strsplit(s, '.', fixed=T)[[1]][1])
}
simple_sample = function(s) {
    sn = strsplit(s, '_', fixed=T)[[1]]
    sn = sn[length(sn)]
    return(sn)
}

rownames(counts) = unlist(lapply(rownames(counts), gencode2ensembl))
colnames(counts) = unlist(lapply(colnames(counts), simple_sample))

In [5]:
# load in sex genes
mapping = read.table('../16p12.2_rnaseq_analysis/family_based_analysis3/gene_map.tsv', sep='\t', header=TRUE, stringsAsFactors = F)
rownames(mapping)   = mapping$Name
sex_genes = scan('../16p12.2_rnaseq_analysis/family_based_analysis3/gtex_filter_sex_diff2.list', what="", sep="\n")
xgenes = as.character(mapping[mapping$chromosome == 'X' ,]$ensembl)
ygenes = as.character(mapping[mapping$chromosome == 'Y' ,]$ensembl)
# length(low_expressed_genes)
length(xgenes)
length(ygenes)
length(sex_genes)

[1] 2358

[1] 491

[1] 340

In [6]:
# Filter out sex genes
dim(counts)
counts=counts[!(rownames(counts) %in% xgenes),]
dim(counts)
counts=counts[!(rownames(counts) %in% ygenes),]
dim(counts)
counts=counts[!(rownames(counts) %in% sex_genes),]
dim(counts)

[1] 56202    96

[1] 53844    96

[1] 53353    96

[1] 53013    96

In [7]:
kinship = read.table('../gemma/output/relatedness_matrix_by_replicate.cXX.txt', sep='\t')
colnames(kinship) = fam$V2
rownames(kinship) = fam$V2

In [8]:
samps = colnames(counts)

pheno = pheno[samps,]
counts = counts[, samps]
kinship = kinship[samps, samps]

status = pheno$status3
status = as.numeric(as.factor(status))
status = 2 - status

In [9]:
dim(counts)

[1] 53013    96

In [10]:
# counts = head(counts, n=1000)

In [11]:
fit = pqlseq(RawCountDataSet=counts, 
             Phenotypes=status, 
             RelatednessMatrix=kinship,
             fit.model="PMM")

## number of total individuals:  96 
## number of total genes/sites:  36443 
## number of adjusted covariates:  0 


Warning message in pqlseq(RawCountDataSet = counts, Phenotypes = status, RelatednessMatrix = kinship, :
“PQLseq::the relatedness matrix is singular, it has been modified!”


ERROR: Error in isOpen(con): invalid connection


In [14]:
write.table(fit, 'output/pqlseq_by_replicate_no_sex.tsv', sep='\t')

In [12]:
dim(fit)

[1] 36443     7